In [1]:
import pandas as pd
fp = "../data/incident_event_log.csv"
df = pd.read_csv(fp)
df.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [2]:
support_cols =["number","assignment_group", "assigned_to"]
df[support_cols].head(25)

,number,assignment_group,assigned_to
0,INC0000045,Group 56,?
1,INC0000045,Group 56,?
2,INC0000045,Group 56,?
3,INC0000045,Group 56,?
4,INC0000047,Group 70,Resolver 89
5,INC0000047,Group 24,Resolver 31
6,INC0000047,Group 24,Resolver 31
7,INC0000047,Group 24,Resolver 31
8,INC0000047,Group 24,Resolver 31
9,INC0000047,Group 24,Resolver 31


In [3]:
df["incident_ID"] = "incident-" + df["number"]
df["sgid"] = "sg-" + df["assignment_group"]
df["vendorid"] = "vendor-" + df["vendor"]
df["customer"] = "customer-" + df["opened_by"]

In [4]:
df = df.groupby("incident_ID").last()

In [5]:
support_cols =["assignment_group", "assigned_to"]
df_sg = df[support_cols]
df_sg = df_sg.replace(to_replace = '?', value = 'UNKNOWN')
df_sg = df_sg.astype("category")
cols = df_sg.columns.tolist()
vocab_size = 0
for c in cols:
    print("Num unique vals for category " + str(c) + " = " + str(df_sg[c].nunique()))
    vocab_size += df_sg[c].nunique()
print("Vocab size: %s" % vocab_size)

Num unique vals for category assignment_group = 71
Num unique vals for category assigned_to = 221
Vocab size: 292


In [6]:
df_sg["assignment_group"].value_counts()

Group 70    9444
UNKNOWN     2157
Group 25    1243
Group 39    1199
Group 24    1060
            ... 
Group 18       2
Group 77       1
Group 7        1
Group 80       1
Group 41       1
Name: assignment_group, Length: 71, dtype: int64

In [7]:
import numpy as np
df_sg_t = pd.get_dummies(data=df_sg, dtype= np.single)
df_sg_t["sgid"] = df["sgid"].values
del df_sg

In [8]:
sg_mapping = {index: i for i, index in enumerate(df_sg_t.sgid.unique())}

In [9]:
numeric = ['sys_mod_count', 'reopen_count']
dfn = df[numeric]
dcols = []
for col in numeric:
    dlabel = 'D_' + col
    labels = [dlabel +'_' + str(c) for c in range(5)]
    dcols.append(dlabel)
    dfn[dlabel] = pd.qcut(dfn[col].rank(method='first'),5, labels = labels, duplicates = 'drop')
    
attributes = df.columns.tolist()
remove = [ 'made_sla', 'opened_at', 'resolved_at','sys_created_at', 'caller_id', 'closed_at',\
          'notify', 'sys_updated_by','sys_created_by', 'number', 'sys_updated_at', 'reassigned', 'reassignment_count' ]
exclude = remove + numeric
keep = list(set(attributes) - set(exclude)) 

/home/admin2/anthony_pyGAdapter_test/pygtest/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
df_incident = df[keep]
df_incident = df_incident.replace(to_replace = '?', value = 'UNKNOWN')
df_incident =  pd.concat([df_incident, dfn[dcols]], axis = 1)
df_incident = df_incident.astype("category")
df['made_sla'] = df['made_sla'].map({True: 1, False: 0})
df_incident_t = pd.get_dummies(data=df_incident, dtype=np.single)
df = df.reset_index()
del df_incident

In [11]:
df['reassigned'] = df['reassignment_count'].apply(lambda x: 0 if x == 0 else 1)
df['reassigned'].value_counts()

0    13549
1    11369
Name: reassigned, dtype: int64

In [12]:
df_incident_t["reassigned"] = df["reassigned"].values

In [13]:
df_incident_t["incident_ID"] = df["incident_ID"].values
#df_incident.set_index("incident_ID", inplace=True)

In [14]:
incident_mapping = {index: i for i, index in enumerate(df_incident_t.incident_ID.unique())}

In [15]:
# cols = df_incident.columns.tolist()
# vocab_size = 0
# for c in cols:
#     print("Num unique vals for category " + str(c) + " = " + str(df_incident[c].nunique()))
#     vocab_size += df_incident[c].nunique()
# print("Vocab size: %s" % vocab_size)

In [16]:
cust_cols = ["opened_by"]
df_cust = df[cust_cols]
df_cust = df_cust.replace(to_replace = '?', value = 'UNKNOWN')
df_cust = df_cust.astype("category")
cols = df_cust.columns.tolist()
vocab_size = 0
for c in cols:
    print("Num unique vals for category " + str(c) + " = " + str(df_cust[c].nunique()))
    vocab_size += df_cust[c].nunique()
print("Vocab size: %s" % vocab_size)

Num unique vals for category opened_by = 208
Vocab size: 208


In [17]:
df_cust_t = pd.get_dummies(data=df_cust, dtype=np.single)
del df_cust

In [18]:
df_cust_t["customer"] = df["customer"].values
#df_cust.set_index("customer", inplace=True)

In [19]:
cust_mapping = {index: i for i, index in enumerate(df_cust_t.customer.unique())}

In [20]:
vendor_cols=["vendor"]
df_vendor = df[vendor_cols]
df_vendor = df_vendor.replace(to_replace = '?', value = 'UNKNOWN')
df_vendor = df_vendor.astype("category")
cols = df_vendor.columns.tolist()
vocab_size = 0
for c in cols:
    print("Num unique vals for category " + str(c) + " = " + str(df_vendor[c].nunique()))
    vocab_size += df_vendor[c].nunique()
print("Vocab size: %s" % vocab_size)

Num unique vals for category vendor = 3
Vocab size: 3


In [21]:
df_vendor_t = pd.get_dummies(data=df_vendor, dtype=np.single)
df_vendor_t["vendorid"] = df["vendorid"].values
#df_vendor.set_index("vendorid", inplace=True)

In [22]:
vendor_mapping = {index: i for i, index in enumerate(df_vendor_t.vendorid.unique())}

In [23]:
ecols = ["incident_ID", "sgid", "customer", "vendorid"]
df_edges = df[ecols]

In [24]:
import torch
iedges = [incident_mapping[i] for i in df_edges["incident_ID"].values]
sgedges = [sg_mapping[i] for i in df_edges["sgid"].values]
cedges = [cust_mapping[i] for i in df_edges["customer"].values]
vedges = [vendor_mapping[i] for i in df_edges["vendorid"].values]


In [25]:
from torch_geometric.utils import to_undirected
edge_inc_cust = torch.tensor([[iedges[i], cedges[i]] for i in range(df_edges.shape[0])], dtype=torch.long)
#edge_inc_cust_index = to_undirected(edge_inc_cust_index)
edge_inc_cust_index = edge_inc_cust.t().contiguous()
edge_inc_sg = torch.tensor([[iedges[i], sgedges[i]] for i in range(df_edges.shape[0])], dtype=torch.long)
edge_inc_sg_index = edge_inc_sg.t().contiguous()
#edge_inc_sg_index = to_undirected(edge_inc_sg_index)
edge_inc_vendor= torch.tensor([[iedges[i], vedges[i]] for i in range(df_edges.shape[0])], dtype=torch.long)
edge_inc_vendor_index = edge_inc_vendor.t().contiguous()
#edge_inc_vendor_index = to_undirected(edge_inc_vendor_index)

In [26]:
df_edges

,incident_ID,sgid,customer,vendorid
0,incident-INC0000045,sg-Group 56,customer-Opened by 8,vendor-?
1,incident-INC0000047,sg-Group 24,customer-Opened by 397,vendor-?
2,incident-INC0000057,sg-Group 70,customer-Opened by 8,vendor-?
3,incident-INC0000060,sg-Group 25,customer-Opened by 180,vendor-?
4,incident-INC0000062,sg-Group 23,customer-Opened by 180,vendor-?
...,...,...,...,...
24913,incident-INC0120304,sg-Group 64,customer-Opened by 8,vendor-?
24914,incident-INC0120319,sg-Group 70,customer-Opened by 508,vendor-?
24915,incident-INC0120495,sg-Group 24,customer-Opened by 14,vendor-?
24916,incident-INC0120835,sg-Group 31,customer-Opened by 12,vendor-?


In [27]:
from torch_geometric.data import HeteroData

In [28]:
data = HeteroData()

In [29]:
inc_x = df_incident_t.columns.tolist()
inc_x.remove("reassigned")
inc_x.remove("incident_ID")
df_inc_x = df_incident_t[inc_x]
sg_x = df_sg_t.columns.tolist()
sg_x.remove("sgid")
cx = df_cust_t.columns.tolist()
cx.remove("customer")
vx = df_vendor_t.columns.tolist()
vx.remove("vendorid")

In [30]:
import torch
torch.from_numpy(df_inc_x.values).shape

torch.Size([24918, 2445])

In [31]:
df_inc_x.head()

,resolved_by_Resolved by 10,resolved_by_Resolved by 100,resolved_by_Resolved by 101,resolved_by_Resolved by 102,resolved_by_Resolved by 103,resolved_by_Resolved by 104,resolved_by_Resolved by 105,resolved_by_Resolved by 106,resolved_by_Resolved by 107,resolved_by_Resolved by 108,...,D_sys_mod_count_D_sys_mod_count_0,D_sys_mod_count_D_sys_mod_count_1,D_sys_mod_count_D_sys_mod_count_2,D_sys_mod_count_D_sys_mod_count_3,D_sys_mod_count_D_sys_mod_count_4,D_reopen_count_D_reopen_count_0,D_reopen_count_D_reopen_count_1,D_reopen_count_D_reopen_count_2,D_reopen_count_D_reopen_count_3,D_reopen_count_D_reopen_count_4
incident_ID,,,,,,,,,,,,,,,,,,,,,
incident-INC0000045,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
incident-INC0000047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
incident-INC0000057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
incident-INC0000060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
incident-INC0000062,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [32]:
data["incident"].x = torch.from_numpy(df_inc_x.values )
data["incident"].y = torch.from_numpy(df_incident_t["reassigned"].values)
data["support_group"].x = torch.from_numpy(df_sg_t[sg_x].values)
data["customer"].x = torch.from_numpy(df_cust_t[cx].values)
data["vendor"].x = torch.from_numpy(df_vendor_t[vx].values)
data["incident", "reported_by", "customer"].edge_index = edge_inc_cust_index
data["incident", "worked_by", "support_group"].edge_index = edge_inc_sg_index
data["incident", "requires_support", "vendor"].edge_index  = edge_inc_vendor_index

In [33]:
data

HeteroData(
  incident={
    x=[24918, 2445],
    y=[24918]
  },
  support_group={ x=[24918, 292] },
  customer={ x=[24918, 208] },
  vendor={ x=[24918, 3] },
  (incident, reported_by, customer)={ edge_index=[2, 24918] },
  (incident, worked_by, support_group)={ edge_index=[2, 24918] },
  (incident, requires_support, vendor)={ edge_index=[2, 24918] }
)

In [34]:
import torch_geometric.transforms as T

t1 = T.ToUndirected()
t2 = T.RandomNodeSplit(num_val=0.1, num_test=0.1)

In [35]:
data = t1(data)
data = t2(data)
data

HeteroData(
  incident={
    x=[24918, 2445],
    y=[24918],
    train_mask=[24918],
    val_mask=[24918],
    test_mask=[24918]
  },
  support_group={ x=[24918, 292] },
  customer={ x=[24918, 208] },
  vendor={ x=[24918, 3] },
  (incident, reported_by, customer)={ edge_index=[2, 24918] },
  (incident, worked_by, support_group)={ edge_index=[2, 24918] },
  (incident, requires_support, vendor)={ edge_index=[2, 24918] },
  (customer, rev_reported_by, incident)={ edge_index=[2, 24918] },
  (support_group, rev_worked_by, incident)={ edge_index=[2, 24918] },
  (vendor, rev_requires_support, incident)={ edge_index=[2, 24918] }
)

In [36]:
import torch
import torch.nn
import torch
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, Linear, SAGEConv

import torch
import torch.nn
from torch_geometric.nn import SAGEConv


class GNN(torch.nn.Module):
    def __init__(self, num_hidden_layers, hidden_channels, out_channels):
        super().__init__()

        self.hidden = None
        if num_hidden_layers > 1:
            self.hidden = torch.nn.ModuleList()
            for _ in range(num_hidden_layers - 1):
                self.hidden.append(SAGEConv((-1, -1), hidden_channels))
        self.conv_final = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        if self.hidden:
            for hl in self.hidden:
                x = hl(x, edge_index).relu()
        x = torch.sigmoid(self.conv_final(x, edge_index))
        return x
 


In [37]:
from torch_geometric.nn import to_hetero
model = GNN(num_hidden_layers=2, hidden_channels=64, out_channels=2)
model = to_hetero(model, data.metadata(), aggr="sum")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data, model = data.to(device), model.to(device)


In [38]:
with torch.no_grad():  # Initialize lazy modules.
    out = model(data.x_dict, data.edge_index_dict)

In [39]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

In [44]:
out = model(data.x_dict, data.edge_index_dict)

In [45]:
 mask = data['incident'].train_mask

In [46]:
data['incident'].y[mask]

tensor([0, 1, 0,  ..., 0, 1, 1])

In [49]:
out['incident'][mask]

tensor([[0.4376, 0.4393],
        [0.4420, 0.4176],
        [0.4594, 0.4391],
        ...,
        [0.4484, 0.4227],
        [0.4480, 0.4389],
        [0.4201, 0.4191]], grad_fn=<IndexBackward0>)

In [50]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x_dict, data.edge_index_dict)
    mask = data['incident'].train_mask
    loss = F.cross_entropy(out['incident'][mask], data['incident'].y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)

In [57]:
def test():
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict)['incident'].argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask', 'test_mask']:
        mask = data['incident'][split]
        acc = (pred[mask] == data['incident'].y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs


In [59]:
for epoch in range(1, 101):
    loss = train()
    train_acc, val_acc, test_acc = test()
    if epoch % 5 ==0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
              f'Val: {val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 005, Loss: 0.4865, Train: 0.8284, Val: 0.8278, Test: 0.8206
Epoch: 010, Loss: 0.4721, Train: 0.8425, Val: 0.8383, Test: 0.8351
Epoch: 015, Loss: 0.4601, Train: 0.8552, Val: 0.8423, Test: 0.8471
Epoch: 020, Loss: 0.4513, Train: 0.8650, Val: 0.8487, Test: 0.8535
Epoch: 025, Loss: 0.4465, Train: 0.8701, Val: 0.8519, Test: 0.8527
Epoch: 030, Loss: 0.4429, Train: 0.8753, Val: 0.8523, Test: 0.8511
Epoch: 035, Loss: 0.4404, Train: 0.8759, Val: 0.8511, Test: 0.8515
Epoch: 040, Loss: 0.4386, Train: 0.8793, Val: 0.8543, Test: 0.8519
Epoch: 045, Loss: 0.4382, Train: 0.8751, Val: 0.8571, Test: 0.8596
Epoch: 050, Loss: 0.4367, Train: 0.8801, Val: 0.8624, Test: 0.8600
Epoch: 055, Loss: 0.4333, Train: 0.8872, Val: 0.8624, Test: 0.8571
Epoch: 060, Loss: 0.4303, Train: 0.8904, Val: 0.8652, Test: 0.8591
Epoch: 065, Loss: 0.4289, Train: 0.8921, Val: 0.8692, Test: 0.8688
Epoch: 070, Loss: 0.4273, Train: 0.8911, Val: 0.8672, Test: 0.8716
Epoch: 075, Loss: 0.4236, Train: 0.8942, Val: 0.8716, Test: 0.

In [56]:
 model(data.x_dict, data.edge_index_dict)['incident'].argmax(dim=-1)

tensor([0, 0, 0,  ..., 0, 0, 0])